In [32]:
import pandas as pd
import requests
import ckanapi
import math
import re
import os
import json
import datetime

loading_folder = os.getcwd()
setting_file = loading_folder+'\\..\\setting.json'
with open(setting_file) as f:
    setting = json.load(f)


In [33]:
## Genral setting
env = 'ckan_qa'         # ckan_qa|ckan_dev|ckan_prod
package_name='daily-shelter-occupancy'
res_name='DSO 2020'
fileType = 'CSV'        # CVS|Excel use first tab if it's excel
fileURL = 'https://ckanadmin0.intra.qa-toronto.ca/dataset/17356cdc-c67c-4e69-9611-fabe21e80c48/resource/e72e8a4c-08b1-4885-897b-21b97e02ce60/download/daily-shelter-occupancy-2020.csv.csv'

In [34]:
df = pd.read_csv(fileURL,  na_values=['_'])


In [35]:
fields = []
for field, dtype in df.dtypes.iteritems():
    ckan_type_map = {"int64": "int", "object": "text", "float64": "float"}
    fields.append({"type": ckan_type_map[dtype.name], "id": field})

In [37]:
ckan_connect = setting[env]
ckan = ckanapi.RemoteCKAN(ckan_connect['address'], ckan_connect['apikey'])

res_id=None
try:
    package=ckan.action.package_show(id=package_name)
    resources=package.pop('resources')
    for r1 in resources:
        if r1['name']==res_name: # and r1['datastore_active']:
            res_id = r1['id']
            ckan.action.datastore_delete(id=res_id, filters={}, force=True)
            now = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")
            ckan.action.resource_patch(id=res_id, datastore_active=True, is_preview=True, last_refreshed=now)
            ckan.action.package_patch(id=package['id'], last_refreshed=now)
            ds=ckan.action.datastore_search(id=res_id, limit=1)
            fields=ds['fields'][1:]
            print('Existing Resource:', res_id, 'old records deleted')
            break
except:
    print('Error finding package')

if not res_id:
    try:
        r = {'package_id': package['id'], 'format': 'CSV', 'extract_job': 'CSV-CKAN','name': res_name, 'datastore_active': True,'is_preview': True}
        res = ckan.action.datastore_create( resource=r, fields=fields, primary_key=None, force=True)
        res_id = res['resource_id']
        print('Created new resource', res_id)
    except:
        print('Error creating resource')


Existing Resource: 95c3f0d7-f1f3-46b4-87cf-ffb1f068d0f3 old records deleted


In [38]:
i=0
while i < len(df):
    rec=[ { k:v for k,v in r.items() if (v == v) and (v is not None) } for r in df[i:i+1000].to_dict(orient="records") ]
    try:
        print('Try:',i,i+1000)
        progress_insert=ckan.action.datastore_upsert(id=res_id, records=rec, force=True, method='insert')
    except Exception as e:
        print('Something is wrong:', e)
        break
    i=i+1000

if i >= len(df):
    print('Data loading completed:', len(df))
else:
    print('Data loading aborted')

Try: 0 1000
Try: 1000 2000
Try: 2000 3000
Try: 3000 4000
Try: 4000 5000
Try: 5000 6000
Try: 6000 7000
Try: 7000 8000
Try: 8000 9000
Try: 9000 10000
Try: 10000 11000
Try: 11000 12000
Try: 12000 13000
Try: 13000 14000
Try: 14000 15000
Try: 15000 16000
Try: 16000 17000
Try: 17000 18000
Try: 18000 19000
Try: 19000 20000
Try: 20000 21000
Try: 21000 22000
Try: 22000 23000
Try: 23000 24000
Try: 24000 25000
Try: 25000 26000
Try: 26000 27000
Try: 27000 28000
Try: 28000 29000
Try: 29000 30000
Try: 30000 31000
Try: 31000 32000
Try: 32000 33000
Try: 33000 34000
Try: 34000 35000
Try: 35000 36000
Try: 36000 37000
Try: 37000 38000
Try: 38000 39000
Try: 39000 40000
Try: 40000 41000
Try: 41000 42000
Data loading completed: 41061
